# <span style="color:blue">Homework 2</span>
## Finance 372 - Prof Travis Johnson

### Due 11:59pm on Friday 2/17/23

**Videos**: Programming skills you'll need for this assignment are covered in videos 9 and 10 on the YouTube channel https://www.youtube.com/channel/UCm8Lrbs8DkIZ8OHa0VpK2hQ

**Rules**: you must complete this assignment on your own but are allowed to consult lecture notes, the class YouTube channel, other internet resources, and other students via the Discord Channel. If you are using code copies from a website, cite that website. You are still responsible for the quality of the code submitted - it's *your* fault if you copy an unreliable resource

**Deliverables**: submit an .html copy (File -> Export Notebook As -> Export Notebook to HTML) and your .ipynb file via Canvas upload. The HTML file must be readable and include all the output you want us to grade. We will **not** run your code for you, and you will **not** receive credit for solutions that are in code format but do not have readable output in the HTML file.

## Solution by: <span style="color:orange">Siyuan Yang Sy9734</span>

This Homework will be focused on updating and expanding the following `positions_df`, which is a snapshot of all positions currently in the portfolio, lumping them together by identifier. So if you bought 100 shares ABC on 11/1 and another 150 shares on 12/1, and have made no other trades, current_portfolio will have a row with `security_id = ABC`, `position = 250`.
       
positions_df is meant to match what you observe on your brokerage account screen. In backtests, we keep track of this ourselves. In live trading, this would be pulled from the brokerage.

To get started, we'll need to create `positions_df` from scratch instead of importing it from a file. Syntax for creating a DataFrame from scratch (instead of reading from a file):
```python
df = pd.DataFrame({'column1_name': [row1col1, row2col1, ...],
                   'column2_name': [row1col2, row2col2, ...],
                       ...
                   'columnM_name': [row1colM, row2colM, ...],
                 })
```

If we want to create empty columns with specified dtypes, we can replace `[row1col1, row2col1, ...]` with `pd.Series([], dtype='type')` (https://stackoverflow.com/questions/36462257/create-empty-dataframe-in-pandas-specifying-column-types)

For more info, see https://www.geeksforgeeks.org/different-ways-to-create-pandas-dataframe/

In [1307]:
import pandas as pd
import numpy as np

# Positions DataFrame, with one row per security_id. Contains a snapshot of all positions currently in the portfolio

# First, Create an empty DataFrame with column names but specified dtypes.
# This way when we append new rows we can be sure the dtype stays consistent & all these columns remain
# The columns are:
# security_id: security identifier such as ticker or permno or optionid whatever unique identifier for the position
# quantity: number of units in portfolio, can be positive or negative. Never 0 (zero positions removed from this DataFrame unless its cash)
# average_cost: average cost of units in portfolio
# current_price: current price of security
# market_value: current market value of position
# init_margin: initial margin requirement of position
# main_margin: maintenance margin requirement of position
positions_df = pd.DataFrame({'security_id': pd.Series([], dtype='object'), 
                             'quantity': pd.Series([], dtype='float'), 
                             'average_cost': pd.Series([], dtype='float'), 
                             'current_price': pd.Series([], dtype='float'), 
                             'market_value': pd.Series([], dtype='float'), 
                             'init_margin': pd.Series([], dtype='float'),
                             'maint_margin': pd.Series([], dtype='float'), 
                            })

# Cash row for the dataframe. Quantity starts at zero, current_price and average_cost always 1
# Will set market_value and init_margin later
cash_df = pd.DataFrame({'security_id': ['cash'], 'quantity': [0], 'average_cost': [1], 'current_price': [1]})

positions_df = positions_df.append(cash_df)
positions_df

/var/folders/tg/ntk053s545b13rybq511zbjh0000gn/T/ipykernel_51441/3685972765.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  positions_df = positions_df.append(cash_df)


,security_id,quantity,average_cost,current_price,market_value,init_margin,maint_margin
0,cash,0.0,1.0,1.0,NaN,NaN,NaN


## <span style="color:blue">Task 1</span>: Create an `add_cash` function, and test it by adding $1000 cash and displaying the updated dataframe

In [1308]:
# Solution to Task 1

# Function add_cash accepts as input:
# 1) df: a dataframe with security_id and quantity columns
# 2) cash_to_add: a cash amount that can be positive or negative
# and adds this amount to the 'quantity' column of any row with 'security_id'='cash' in df
# it doesn't return anything
def add_cash(df, cash_to_add):
    df.loc[df['security_id'] == 'cash', 'quantity'] += cash_to_add
    return df
    



##### Run this cell **without** editing it, the output should show positions_df with cash quantity = 1000

In [1309]:
add_cash(df=positions_df, cash_to_add=+1000)
positions_df

,security_id,quantity,average_cost,current_price,market_value,init_margin,maint_margin
0,cash,1000.0,1.0,1.0,NaN,NaN,NaN


## <span style="color:blue">Task 2</span>: Load the transactions in `transactions1.csv` and adjust the quantity column to be negative for sells and positive for buys

In [1310]:
# Solution to Task 2
import pandas as pd
transactions_df = pd.read_csv('transactions1.csv')
transactions_df.loc[transactions_df['direction'] == 'sell', 'quantity'] *= -1


##### Run this cell **without** editing it, the output should show a DataFrame with negative quantities for sells and positive for buys

In [1311]:
transactions_df

,security_id,direction,quantity,average_price
0,12345,buy,100,25.18
1,TWTR,sell,-200,64.18
2,KO,buy,50,48.80


## <span style="color:blue">Task 3</span>: Rename the average_price column in `transactions_df` to average_cost so it matches the column in `positions_df`, remove the direction column

In [1312]:
# Solution to Task 3

transactions_df = transactions_df.rename(columns={'average_price': 'average_cost'})
transactions_df = transactions_df.drop(columns=['direction'])

##### Run this cell **without** editing it, the output should show columns for security_id, average_cost, and quantity (but not direction)

In [1313]:
transactions_df

,security_id,quantity,average_cost
0,12345,100,25.18
1,TWTR,-200,64.18
2,KO,50,48.80


## <span style="color:blue">Task 4</span>: Append the transactions DataFrame to `positions_df`, and use the `add_cash` function to adjust the cash balance to reflect these transactions

Remember, cash balance goes up when you sell and down when you buy

In [1314]:
# Solution to Task 4

positions_df = pd.concat([positions_df,transactions_df])
multiply = transactions_df.loc[:,'quantity'] * transactions_df.loc[:,'average_cost']
add_cash(df=positions_df,cash_to_add= -1* multiply.sum())

,security_id,quantity,average_cost,current_price,market_value,init_margin,maint_margin
0,cash,8878.0,1.00,1.0,NaN,NaN,NaN
0,12345,100.0,25.18,NaN,NaN,NaN,NaN
1,TWTR,-200.0,64.18,NaN,NaN,NaN,NaN
2,KO,50.0,48.80,NaN,NaN,NaN,NaN


##### Run this cell **without** editing it, the output should show rows for each of the new securities, and an updated cash balance

In [1315]:
positions_df


,security_id,quantity,average_cost,current_price,market_value,init_margin,maint_margin
0,cash,8878.0,1.00,1.0,NaN,NaN,NaN
0,12345,100.0,25.18,NaN,NaN,NaN,NaN
1,TWTR,-200.0,64.18,NaN,NaN,NaN,NaN
2,KO,50.0,48.80,NaN,NaN,NaN,NaN


## <span style="color:blue">Task 5</span>: Write a `process_transactions` function that modifies a transactions DataFrame and appends it to the positions DataFrame (as in Tasks 2-4)

Be sure you:
1. Make quantity negative for sells
1. Rename the average_price column to average_cost
1. Remove the direction column 
1. Adjust the cash position to reflect the new transactions
1. Append the new transactions to positions_df and return the combined DataFrame

In [1316]:
# Solution to Task 5

# Function process_transactions takes as input:
# 1) positions_df, a DataFrame with columns: 'security_id', 'quantity', and 'average_cost' (and potentially others)
# 2) transactions_df, a DataFrame with columns: 'security_id', 'direction', 'quantity', and 'average_price'
# It modifies transactions_df so it fits with the conventions of positions_df, adjusts the cash balance in positions_df, and appends to positions_df
# It returns the modified positions_df  
    # Solution to Task 5
def process_transactions(positions_df, transactions_df):
    # make a copy of the original positions_df
    modified_positions_df = positions_df.copy()
    #1.Make quantity negative for sells
    transactions_df.loc[transactions_df['direction'] == 'sell', 'quantity'] *= -1
    #2.rename the average_price column
    transactions_df.rename(columns={'average_price': 'average_cost'},inplace=True)
    #3.Remove the direction column
    transactions_df.drop(columns=['direction'],inplace=True)
    #4.adjust cash position
    multi = transactions_df.loc[:,'quantity'] * transactions_df.loc[:,'average_cost']
    add_cash(df=modified_positions_df,cash_to_add=(multi*-1).sum())
    #5.Appending
    modified_positions_df = pd.concat([modified_positions_df, transactions_df])

    return modified_positions_df




##### Run this cell **without** editing it, the output should show a DataFrame with all five transactions and an updated cash balance

In [1317]:
transactions2_df = pd.read_csv('transactions2.csv')
positions_df = process_transactions(positions_df, transactions2_df)

positions_df

,security_id,quantity,average_cost,current_price,market_value,init_margin,maint_margin
0,cash,8703.0,1.00,1.0,NaN,NaN,NaN
0,12345,100.0,25.18,NaN,NaN,NaN,NaN
1,TWTR,-200.0,64.18,NaN,NaN,NaN,NaN
2,KO,50.0,48.80,NaN,NaN,NaN,NaN
0,12345,100.0,28.19,NaN,NaN,NaN,NaN
1,KO,-50.0,52.88,NaN,NaN,NaN,NaN


## <span style="color:blue">Task 6</span>: Group and combine positions with the same security id to create a new `grouped_positions_df`
1. Combine quantity as sum of quantity across all positions with the same security_id
1. Combine average_cost as the average of average_cost across all positions with the same security_id

So if we have two entries for `security_id='XYZ'`, one with `{'quantity':10, 'average_cost':50}` and the other `{'quantity':20, 'average_cost':60}`, we want to combine them into a single entry with `{'quantity':30, 'average_cost':55}`. It's ok if grouped_positions_df doesn't have the remaining columns (current_price, etc) 

To do this, you will need to use the following syntax:
```python
aggregators = { 'column1_name1': 'agg1_name', 
                'column2_name2': 'agg2_name', 
                'column3_name3': 'agg3_name', 
                ...
              }  
df.groupby('grouping_col').agg(aggregators)
```

and specify the aggregators 'first', 'mean', or 'sum' for each column you want to group and combine (quantity and average_cost in this case).

Note that technically the combined average_cost should be a weighted average of the individual average_costs, but let's ignore this for now

In [1318]:
# Solution to Task 6
aggregators = {'quantity':'sum','average_cost':'mean',}
grouped_positions_df = positions_df.groupby(['security_id']).agg(aggregators)
# grouped_positions_df = grouped_positions_df.sort_values('security_id',ascending=False).reset_index()




##### Run this cell **without** editing it, the output should show a DataFrame with one row for each security_id with quantity and average_cost columns

In [1319]:
grouped_positions_df

,quantity,average_cost
security_id,,
12345,200.0,26.685
KO,0.0,50.840
TWTR,-200.0,64.180
cash,8703.0,1.000


## <span style="color:blue">Task 7</span>: Merge current price and margin requirement data from `prices.csv` into `grouped_positions_df` to create a `position_prices_df`

If you haven't already, you'll need to run `grouped_positions_df = grouped_positions_df.reset_index()` to make the merge work

Make sure your merge keeps all rows originally in the grouped_positions_df even if there isn't a matching security_id in prices.csv

In [1320]:
# Solution to Task 7
prices_df = pd.read_csv('prices.csv')
position_prices_df = grouped_positions_df.merge(prices_df,on='security_id',how='left')

##### Run this cell **without** editing it, the output should show a DataFrame with one row for each security_id with quantity, average_cost, price, init_margin_frac, and maint_margin_frac columns

In [1321]:
position_prices_df

,security_id,quantity,average_cost,price,init_margin_frac,maint_margin_frac
0,12345,200.0,26.685,32.15,1.0,0.75
1,KO,0.0,50.840,58.15,0.4,0.25
2,TWTR,-200.0,64.180,68.22,0.5,0.40
3,cash,8703.0,1.000,NaN,NaN,NaN


## <span style="color:blue">Task 8</span>: Compute a final `positions_df` by starting with `positions_prices_df` and:
1. Removing rows with zero quantity
1. Renaming price to current_price
1. Setting cash's current_price to 1.00
1. Adding and calculating the `market_value`, `init_margin`, and `maint_margin` columns. Note that `_margin_frac` represents the fraction of `market_value`, while `_margin` is the dollar margin requirement. 
1. Removing the extra columns so positions_df has the original columns as defined at the start of this notebook

In [1322]:
# Solution to Task 8
#remove rows with 0 quantity
position_prices_df= position_prices_df[position_prices_df['quantity']!=0]
# Change price to current price
position_prices_df = position_prices_df.rename(columns={'price':'current_price'})
#Setting cash's current_price to 1
position_prices_df.loc[position_prices_df['security_id'] == 'cash', 'current_price'] = position_prices_df.loc[position_prices_df['security_id'] == 'cash', 'current_price'].fillna(1)
#Adding market_value column
position_prices_df.insert(4,'market_value',0)
#Caculating init margin & maint margin
position_prices_df.loc[:,'market_value'] = position_prices_df.loc[:,'quantity'] * position_prices_df.loc[:,'current_price']
position_prices_df.loc[:,'init_margin'] = position_prices_df.loc[:,'init_margin_frac'] * position_prices_df.loc[:,'market_value'].abs()
position_prices_df.loc[:,'maint_margin'] = position_prices_df.loc[:,'maint_margin_frac'] * position_prices_df.loc[:,'market_value'].abs()
position_prices_df.drop(columns=['init_margin_frac','maint_margin_frac'],inplace=True)
positions_df = position_prices_df
#rename position_price_df
# position_prices_df = position_prices_df.rename(columns={'init_margin_frac':'init_margin'})
# position_prices_df = position_prices_df.rename(columns={'maint_margin_frac':'maint_margin'})
# positions_df.loc[:,'current_price':'maint_margin'] = positions_df.loc[:,'current_price':'maint_margin'].fillna(1)




##### Run this cell **without** editing it, the output should show a DataFrame with one row for each security_id and all the original positions_df columns with no NaN values

In [1323]:
positions_df


,security_id,quantity,average_cost,current_price,market_value,init_margin,maint_margin
0,12345,200.0,26.685,32.15,6430.0,6430.0,4822.5
2,TWTR,-200.0,64.180,68.22,-13644.0,6822.0,5457.6
3,cash,8703.0,1.000,1.00,8703.0,NaN,NaN


## <span style="color:blue">Task 9 (Extra Credit)</span> Expand the `process_transactions` function to also automate Tasks 6-8


In [1324]:
# Solution to Task 9

# Function transactions_to_positions takes as input:
# 1) positions_df, a DataFrame with columns: 'security_id', 'quantity', and 'average_cost' (and potentially others)
# 2) transactions_df, a DataFrame with columns: 'security_id', 'direction', 'quantity', and 'average_price'
# 3) prices_df, a DataFrame with columns 'security_id','price','init_margin_frac', 'maint_margin_frac'
#
# It modifies transactions_df so it fits with the conventions of positions_df, adjusts the cash balance in positions_df, appends to positions_df, 
# groups the positions_df by security_id, merges with prices_df
#
# It returns the modified positions_df
def process_transactions(positions_df, transactions_df, prices_df):
    modified_positions_df = positions_df.copy()
    
    # Code from above (Task 5) -------------------------------------------
    modified_positions_df = positions_df.copy()

    #1.Make quantity negative for sells
    transactions_df.loc[transactions_df['direction'] == 'sell', 'quantity'] *= -1
    #2.rename the average_price column
    transactions_df.rename(columns={'average_price': 'average_cost'},inplace=True)
    #3.Remove the direction column
    transactions_df.drop(columns=['direction'],inplace=True)
    #4.adjust cash position
    multi = transactions_df.loc[:,'quantity'] * transactions_df.loc[:,'average_cost']
    add_cash(df=modified_positions_df,cash_to_add=(multi*-1).sum())
    #5.Concatenate
    modified_positions_df = pd.concat([modified_positions_df, transactions_df])

    # New code added (modified from Tasks 6-8) ---------------------------
    # Aggregate
    aggregators = {'quantity':'sum','average_cost':'mean',}
    grouped_positions_df = modified_positions_df.groupby(['security_id']).agg(aggregators)
    position_prices_df = grouped_positions_df.merge(prices_df,on='security_id',how='left')
    position_prices_df= position_prices_df[position_prices_df['quantity']!=0]
    position_prices_df = position_prices_df.rename(columns={'price':'current_price'})
    position_prices_df.loc[position_prices_df['security_id'] == 'cash', 'current_price'] = position_prices_df.loc[position_prices_df['security_id'] == 'cash', 'current_price'].fillna(1)
    position_prices_df.insert(4,'market_value',0)
    position_prices_df.loc[:,'market_value'] = position_prices_df.loc[:,'quantity'] * position_prices_df.loc[:,'current_price']
    position_prices_df.loc[:,'init_margin'] = position_prices_df.loc[:,'init_margin_frac'] * position_prices_df.loc[:,'market_value'].abs()
    position_prices_df.loc[:,'maint_margin'] = position_prices_df.loc[:,'maint_margin_frac'] * position_prices_df.loc[:,'market_value'].abs()
    position_prices_df.drop(columns=['init_margin_frac','maint_margin_frac'],inplace=True)
    positions_df = position_prices_df
    return positions_df

In [1325]:
# Run this cell to test your solution to Task 9. It should produce the same final dataframe you had in Task 8

# First, Create a new empty positions DataFrame
positions_df = pd.DataFrame({'security_id': pd.Series([], dtype='object'), 
                             'quantity': pd.Series([], dtype='float'), 
                             'average_cost': pd.Series([], dtype='float'), 
                             'current_price': pd.Series([], dtype='float'), 
                             'market_value': pd.Series([], dtype='float'), 
                             'init_margin': pd.Series([], dtype='float'),
                             'maint_margin': pd.Series([], dtype='float'), 
                            })

# Cash row for the dataframe. Quantity starts at zero, current_price and average_cost always 1
# Will set market_value and init_margin later
cash_df = pd.DataFrame({'security_id': ['cash'], 'quantity': [0], 'average_cost': [1], 'current_price': [1]})
positions_df = positions_df.append(cash_df)
add_cash(df=positions_df,cash_to_add=1000)

# Second, process both sets of transactions
positions_df = process_transactions(positions_df, pd.read_csv('transactions1.csv'), prices_df)
positions_df = process_transactions(positions_df, pd.read_csv('transactions2.csv'), prices_df)

positions_df

/var/folders/tg/ntk053s545b13rybq511zbjh0000gn/T/ipykernel_51441/3093445194.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  positions_df = positions_df.append(cash_df)


,security_id,quantity,average_cost,current_price,market_value,init_margin,maint_margin
0,12345,200.0,26.685,32.15,6430.0,6430.0,4822.5
2,TWTR,-200.0,64.180,68.22,-13644.0,6822.0,5457.6
3,cash,8703.0,1.000,1.00,8703.0,NaN,NaN
